

# COTTON LEAF DISEASE PREDICTION - TRANSFER LEARNING USING INCEPTIONV3





In [15]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [17]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/My Drive/data/train'
valid_path = '/content/drive/My Drive/data/val'

In [18]:
inceptionV3 = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 0s 0us/step


In [19]:
for layer in inceptionV3.layers:
    layer.trainable = False

In [20]:
folders = glob('/content/drive/My Drive/data/train/*')

In [21]:
x = Flatten()(inceptionV3.output)

In [22]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=inceptionV3.input, outputs=prediction)

In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [24]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [25]:
train_data = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_data = ImageDataGenerator(rescale = 1./255)

In [26]:
training_set = train_data.flow_from_directory('/content/drive/My Drive/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [27]:
test_set = test_data.flow_from_directory('/content/drive/My Drive/data/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 324 images belonging to 4 classes.


In [28]:
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
61/61 [==============================] - 544s 9s/step - loss: 5.5948 - accuracy: 0.5561 - val_loss: 0.6170 - val_accuracy: 0.8704
Epoch 2/50
61/61 [==============================] - 36s 591ms/step - loss: 0.5790 - accuracy: 0.9032 - val_loss: 0.5084 - val_accuracy: 0.8951
Epoch 3/50
61/61 [==============================] - 37s 599ms/step - loss: 0.8455 - accuracy: 0.8680 - val_loss: 0.2938 - val_accuracy: 0.9444
Epoch 4/50
61/61 [==============================] - 37s 604ms/step - loss: 0.3127 - accuracy: 0.9407 - val_loss: 0.3411 - val_accuracy: 0.9414
Epoch 5/50
61/61 [==============================] - 37s 601ms/step - loss: 0.3836 - accuracy: 0.9339 - val_loss: 0.5419 - val_accuracy: 0.9074
Epoch 6/50
61/61 [==============================] - 37s 600ms/step - loss: 0.4851 - accuracy: 0.9190 - val_loss: 0.4255 - val_accuracy: 0.9228
Epoch 7/50
61/61 [==============================] - 37s 606ms/step - loss: 0.4406 - accuracy: 0.9321 - val_loss: 0.5809 - val_accuracy: 0.9198
E

In [29]:
y_pred = model.predict(test_set)
y_pred

array([[0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 7.7740237e-32],
       [0.0000000e+00, 9.9999893e-01, 1.3221560e-18, 1.1135000e-06],
       [0.0000000e+00, 1.5005598e-37, 1.0000000e+00, 9.4223686e-25]],
      dtype=float32)

In [30]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([1, 0, 0, 2, 3, 1, 2, 2, 2, 0, 1, 1, 1, 2, 2, 1, 3, 1, 0, 1, 1, 3,
       3, 1, 1, 1, 1, 3, 1, 3, 2, 2, 2, 3, 0, 2, 0, 2, 3, 2, 3, 1, 0, 3,
       3, 2, 1, 3, 3, 2, 3, 3, 2, 1, 2, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 2,
       3, 1, 3, 1, 0, 1, 2, 3, 1, 3, 3, 1, 2, 1, 1, 2, 0, 0, 2, 2, 3, 3,
       3, 2, 3, 0, 0, 1, 0, 2, 0, 2, 3, 3, 3, 3, 1, 1, 1, 1, 0, 2, 1, 2,
       3, 1, 0, 3, 3, 3, 2, 1, 3, 1, 3, 0, 2, 2, 0, 2, 2, 1, 3, 3, 2, 2,
       0, 0, 1, 1, 1, 2, 3, 3, 0, 2, 1, 2, 2, 1, 2, 1, 3, 3, 0, 3, 0, 1,
       2, 0, 1, 2, 2, 3, 3, 1, 1, 2, 1, 1, 1, 2, 1, 3, 2, 1, 3, 1, 2, 0,
       3, 3, 1, 2, 3, 3, 2, 2, 3, 0, 3, 1, 3, 2, 1, 1, 3, 1, 3, 2, 1, 0,
       2, 2, 1, 1, 3, 1, 2, 3, 2, 0, 3, 0, 2, 3, 2, 1, 2, 2, 0, 3, 3, 1,
       1, 2, 2, 2, 1, 3, 0, 1, 1, 0, 3, 3, 1, 3, 3, 1, 3, 3, 1, 1, 2, 0,
       3, 2, 0, 1, 1, 1, 1, 2, 2, 3, 3, 0, 3, 0, 2, 3, 1, 1, 3, 0, 0, 1,
       1, 3, 2, 3, 2, 2, 1, 1, 3, 0, 0, 2, 3, 1, 2, 1, 1, 1, 3, 0, 1, 3,
       0, 0, 3, 0, 2, 2, 2, 2, 0, 1, 0, 0, 1, 1, 1,

In [32]:
imag = image.load_img('/content/d1.jpg', target_size = (224, 224))
x = image.img_to_array(imag)
x=x/255
test_image = np.expand_dims(x, axis = 0)
result = model.predict(test_image)
result=np.argmax(result, axis=1)
if result==0:
        print("The leaf is diseased cotton leaf")
elif result==1:
        print("The leaf is diseased cotton plant")
elif result==2:
        print("The leaf is fresh cotton leaf")
else:
        print("The leaf is fresh cotton plant")

The leaf is fresh cotton leaf


In [33]:
imag = image.load_img('/content/d2.jpg', target_size = (224, 224))
x = image.img_to_array(imag)
x=x/255
test_image = np.expand_dims(x, axis = 0)
result = model.predict(test_image)
result=np.argmax(result, axis=1)
if result==0:
        print("The leaf is diseased cotton leaf")
elif result==1:
        print("The leaf is diseased cotton plant")
elif result==2:
        print("The leaf is fresh cotton leaf")
else:
        print("The leaf is fresh cotton plant")

The leaf is diseased cotton plant


In [34]:
imag = image.load_img('/content/d3.jpg', target_size = (224, 224))
x = image.img_to_array(imag)
x=x/255
test_image = np.expand_dims(x, axis = 0)
result = model.predict(test_image)
result=np.argmax(result, axis=1)
if result==0:
        print("The leaf is diseased cotton leaf")
elif result==1:
        print("The leaf is diseased cotton plant")
elif result==2:
        print("The leaf is fresh cotton leaf")
else:
        print("The leaf is fresh cotton plant")

The leaf is diseased cotton leaf


In [35]:
imag = image.load_img('/content/d4.jpg', target_size = (224, 224))
x = image.img_to_array(imag)
x=x/255
test_image = np.expand_dims(x, axis = 0)
result = model.predict(test_image)
result=np.argmax(result, axis=1)
if result==0:
        print("The leaf is diseased cotton leaf")
elif result==1:
        print("The leaf is diseased cotton plant")
elif result==2:
        print("The leaf is fresh cotton leaf")
else:
        print("The leaf is fresh cotton plant")

The leaf is fresh cotton plant
